In [14]:
#CRIANDO BIBLIOTECA DM_Util#

#-*- coding: utf-8 -*-
##------------------------------------------------------------------------
## Objective: Funções gerais
## Autor: Prof. Roberto Ãngelo
## Aluna: Rosane Ananias
##------------------------------------------------------------------------

# Bibliotecas padrão
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from scipy.stats import ks_2samp
import pickle


#----------------------------------------------
#-- Função para cálculo de KS2
#----------------------------------------------
def KS2(y, y_pred):
    df_ks2 = pd.DataFrame([x for x in y_pred], columns=['REGRESSION_RLog'])
    df_ks2['ALVO'] = [x for x in y]
    return ks_2samp(df_ks2.loc[df_ks2.ALVO==0,"REGRESSION_RLog"], df_ks2.loc[df_ks2.ALVO==1,"REGRESSION_RLog"])[0]
#----------------------------------------------

#--------------------------------------------------------
#-- Funções mostrar fórmula das regressões em formato SQL Server
#--------------------------------------------------------
def LogisticFormulaSQL(model, X):
    print('Regressão Logística')
    print('SCORE = ROUND(1/(1 + exp(-(       ' + str(model.intercept_[0]))
    for i in range(0, len(model.coef_[0])):
        print('             + {:30}     *     {:.6}'.format(X.columns[i], str(model.coef_[0,i])))
    print('             ))) * 100,2) 	')

def LinearFormulaSQL(model, X):
    print('Regressão Linear')
    print('SCORE = ' + str(model.intercept_))
    for i in range(0, len(model.coef_)):
        print('             + {:30}     *     {:.6}'.format(X.columns[i], str(model.coef_[i])))
#--------------------------------------------------------
        
        
#--------------------------------------------------------
#-- Função para salvar modelo
#--------------------------------------------------------
def SaveModel(model,name):
    File_Model = open(str(name) + '.model', 'wb')
    pickle.dump(model, File_Model)
    File_Model.close()
#--------------------------------------------------------
    
#--------------------------------------------------------
#-- Função para carregar modelo
#--------------------------------------------------------
def OpenModel(name):
    File_Model = open(str(name) + '.model', 'rb')
    model = pickle.load(File_Model)        
    File_Model.close()
    return model
#--------------------------------------------------------
        

In [19]:
# RODANDO O CÓDIGO PARA MINERAÇÃO DOS DADOS E GERAÇÃO DOS MODELOS#
#-*- coding: utf-8 -*-
##------------------------------------------------------------------------
## Case: Marketing
## Autor: Prof. Roberto Angelo
## Aluna: Rosane Ananias
##------------------------------------------------------------------------

# Bibliotecas padrão
import DM_Utils as du
import numpy as np
import pandas as pd


print()
print('----------------     INÍCIO DE CARGA DE DADOS   ------------------------------------')
## Carregando os dados
dataset = pd.read_csv('BASE_MARKETING.TXT',sep='\t') # Separador TAB
print('-----------------     FIM DA CARGA DE DADOS   --------------------------------------')
#print(dataset.describe())

##------------------------------------------------------------
## Separando os conjuntos de treino e teste
## Criando as colunas de "TRN' (Treino) e "TST" (Teste)
##------------------------------------------------------------
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.25, random_state=1233)
train.insert(0,'CONJUNTO','TRN') # Cria coluna de Conjunto com 'TRN' preenchido
test.insert(0,'CONJUNTO','TST')  # Cria coluna de Conjunto com 'TST' preenchido   
dataset = pd.concat([test, train])
 
#EXPORTANDO A BASE-PRE_PROCESSAMENTO
dataset.to_csv('pre_processamento.csv') # Exporta arquivo para análise em outra ferramenta
print('-> DADOS PARA ANÁLISE EXPORTADOS')
# Usar este arquivo para análisar as transformações das colunas e fazer código abaixo
# Ou fazer a análise e tratamento aqui no Python.


#------------------------------------------------------------------------------------------
# Pré-processamento das variáveis de entrada - lIST COMPREHENSIONS
#------------------------------------------------------------------------------------------
# ATRIBUTO VL_TOTAL_INVESTIMENTO_T0
dataset['pre_tem_investimento']   = [1 if np.isnan(x) or x > 0 else 0 for x in dataset['VL_TOTAL_INVESTIMENTO_T0']]

# ATRIBUTO RESTRICAO_INTERNA
dataset['pre_restricao_interna']   = [x for x in dataset['RESTRICAO_INTERNA']]

# ATRIBUTO RISCO
dataset['pre_alto'] = [1 if x=='ALTO' else 0 for x in dataset['RISCO']]
dataset['pre_medio'] = [1 if x=='MEDIO' else 0 for x in dataset['RISCO']]
dataset['pre_baixo'] = [1 if x=='BAIXO' else 0 for x in dataset['RISCO']]

# ATRIBUTO DS_ESTADO_CIVIL
dataset['pre_viuvo'] = [1 if x =='VIUVO' else 0 for x in dataset['DS_ESTADO_CIVIL']]
dataset['pre_solteiro'] = [1 if x =='SOLTEIRO' else 0 for x in dataset['DS_ESTADO_CIVIL']]
dataset['pre_divorciado_nao_informado'] = [1 if x=='DIVORCIADO' or x=='NÃO INFORMADO' else 0 for x in dataset['DS_ESTADO_CIVIL']]

# ATRIBUTO TEM_PRE_APROV_CDC
dataset['pre_tem_pre_aprov_cdc']   = [x for x in dataset['TEM_PRE_APROV_CDC']]

#  ATRIBUTO TIPO_CLIENTE
dataset['pre_tipo_cliente_vip'] = [1 if x =='VIP' else 0 for x in dataset['TIPO_CLIENTE']]
dataset['pre_tipo_cliente_classico'] = [1 if x =='CLÁSSICO' else 0 for x in dataset['TIPO_CLIENTE']]

# ATRIBUTO IDADE
dataset['pre_idade'] = [18 if np.isnan(x) or x < 18 else x for x in dataset['IDADE']] 
dataset['pre_idade'] = [80 if x > 80 else x for x in dataset['pre_idade']] 
dataset['pre_idade'] = [(x-18)/(80-18) for x in dataset['pre_idade']] 

# ATRIBUTO ESCOLARIDADE
dataset['pre_ensino_fundamental'] = [1 if x=='ENSINO FUNDAMENTAL' else 0 for x in dataset['ESCOLARIDADE']] 
dataset['pre_superior'] = [1 if x=='SUPERIOR' else 0 for x in dataset['ESCOLARIDADE']] 
dataset['pre_escolaridade_demais'] = [1 if x!='ENSINO FUNDAMENTAL' and x!='SUPERIOR' else 0 for x in dataset['ESCOLARIDADE']] 

# ATRIBUTO QTDE_PRODUTOS_PF_12
dataset['pre_qtde_produtos_pf_12m'] = [0 if np.isnan(x) or x < 0 else x for x in dataset['QTDE_PRODUTOS_12M']] 
dataset['pre_qtde_produtos_pf_12m'] = [10 if x > 10 else x for x in dataset['pre_qtde_produtos_pf_12m']] 
dataset['pre_qtde_produtos_pf_12m'] = [x/10 for x in dataset['pre_qtde_produtos_pf_12m']] 

###### INCREMENTADO O CÓDIGO CONFORME SOLICITADO NA TERCEIRA AULA (NÃO PRÉ)###


# ATRIBUTO VL_TOTL_REND (X-XMIN)/(X-XMAX)
dataset['pre_vl_totl_rend'] = [0 if np.isnan(x) or x < 0 else x for x in dataset['VL_TOTL_REND']] 
dataset['pre_vl_totl_rend'] = [10000 if x > 10000 else x for x in dataset['pre_vl_totl_rend']] 
dataset['pre_vl_totl_rend'] = [x/10000 for x in dataset['pre_vl_totl_rend']]

# ATRIBUTO CD_SEXO
dataset['pre_cd_sexo']   = [1 if x == 'M' else 0 for x in dataset['CD_SEXO']]

# ATRIBUTO SG_UF -SELEÇÃO BASEADA NA QUANTIDADE DE BONS

dataset['pre_uf_sp']   = [1 if x == 'SP' else 0 for x in dataset['SG_UF']]#1
dataset['pre_uf_rj']   = [1 if x == 'MG' else 0 for x in dataset['SG_UF']]#2
dataset['pre_uf_mg']   = [1 if x == 'RS' else 0 for x in dataset['SG_UF']]#3
dataset['pre_uf_rs']   = [1 if x == 'RJ' else 0 for x in dataset['SG_UF']]#4
dataset['pre_uf_pr']   = [1 if x == 'PR' else 0 for x in dataset['SG_UF']]#5
dataset['pre_uf_pe']   = [1 if x == 'PE' else 0 for x in dataset['SG_UF']]#6
dataset['pre_uf_0']    = [1 if x == '' else 0 for x in dataset['SG_UF']]#DEMAIS

# ATRIBUTO PERFIL 
dataset['pre_investidor']  = [1 if x=='INVESTIDOR' or x=='VAZIO' else 0 for x in dataset['PERFIL']]
dataset['pre_neutro']  = [1 if x=='NEUTRO' else 0 for x in dataset['PERFIL']]
dataset['pre_tomador']  = [1 if x=='TOMADOR' else 0 for x in dataset['PERFIL']]


# ---------------------------------------------------------------------------
# Selecionando as colunasjá pré-processadas
# ---------------------------------------------------------------------------
cols_in =  [   'pre_tem_investimento'
              ,'pre_restricao_interna'
              ,'pre_alto'
              ,'pre_medio'
              ,'pre_baixo'
              ,'pre_viuvo'
              ,'pre_solteiro'
              ,'pre_divorciado_nao_informado'
              ,'pre_tem_pre_aprov_cdc'
              ,'pre_tipo_cliente_vip'
              ,'pre_tipo_cliente_classico'
              ,'pre_idade'
              ,'pre_ensino_fundamental'
              ,'pre_superior'
              ,'pre_escolaridade_demais'
              ,'pre_qtde_produtos_pf_12m'
              ,'pre_vl_totl_rend'
              ,'pre_cd_sexo'
              ,'pre_uf_sp'
              ,'pre_uf_rj'
              ,'pre_uf_mg'
              ,'pre_uf_rs'
              ,'pre_uf_pr'
              ,'pre_uf_pe'
              ,'pre_uf_0'
              ,'pre_investidor'
              ,'pre_neutro'
              ,'pre_tomador'
              ,'ALVO'] # Com ALVO temporariamente

##------------------------------------------------------------
## Separando em dados de treinamento e teste com Oversampling
##------------------------------------------------------------
y = dataset['ALVO']
X = dataset[cols_in] # Com ALVO temporariamente
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 125)

# Replica o número de bons - Oversampling no treino
X_train_1 = X_train.query("ALVO == 1")
X_train = pd.concat([X_train, X_train_1,X_train_1,X_train_1] , sort = False)

# Refaz o y_train
y_train = X_train['ALVO']
#retirar alvo das variáveis de entrada
del X_train['ALVO'] 


#---------------------------------------------------------------------------
## Selecionando Atributos com RFE - Recursive Feature Elimination
#---------------------------------------------------------------------------
# feature extraction
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

##############TESTE_FEATURES
model = LogisticRegression(solver='newton-cg')
selected = RFE(model,step=1,n_features_to_select=16).fit(X_train, y_train)

print()
print('----------------     SELEÇÃO DE VARIÁVEIS--------------------------------------')
print("Num Features: %d" % selected.n_features_)
used_cols = []
for i in range(0, len(selected.support_)):
    if selected.support_[i]: 
        used_cols.append(X_train.columns[i]) 
        print('             -> {:30}     '.format(X_train.columns[i]))
print('-------------------------------------------------------------------------------')

X_train = X_train[used_cols]     # Carrega colunas de entrada selecionadas por RFE
X_test = X_test[used_cols]       # Carrega colunas de entrada selecionadas por RFE
#---------------------------------------------------------------------------
## Ajustando modelos - Aprendizado supervisionado  
#---------------------------------------------------------------------------
# Árvore de decisão com dados de treinamento
from sklearn.tree import DecisionTreeClassifier
#dtree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dtree = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=30,
                       min_weight_fraction_leaf=0.0,
                       random_state=0, splitter='best')
dtree.fit(X_train, y_train)

# Regressão linear com dados de treinamento
from sklearn.linear_model import LinearRegression
LinearReg = LinearRegression(fit_intercept=True)
LinearReg.fit(X_train, y_train)

# Regressão logística com dados de treinamento
from sklearn.linear_model import LogisticRegression
LogisticReg = LogisticRegression(solver='newton-cg')
LogisticReg.fit(X_train, y_train)

#Rede Neural com dados de treinamento
from sklearn.neural_network import MLPClassifier 
RNA = MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=True,
       epsilon=1e-08, hidden_layer_sizes=(25), learning_rate='constant',
       learning_rate_init=0.01, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.25, verbose=False,
       warm_start=False)
RNA.fit(X_train, y_train)


#---------------------------------------------------------------------------
## Salvando os Modelos para Implantar com outro código Python
#---------------------------------------------------------------------------
du.SaveModel(dtree,'dtree')
du.SaveModel(LinearReg,'LinearReg')
du.SaveModel(LogisticReg,'LogisticReg')
du.SaveModel(RNA,'RNA')

#---------------------------------------------------------------------------
## Previsão de treinamento e teste - CLASSIFICAÇÃO
#---------------------------------------------------------------------------
# Árvore de Decisão
y_pred_train_DT = dtree.predict(X_train)
y_pred_test_DT  = dtree.predict(X_test)
# Regressão Linear
y_pred_train_RL = np.array([1 if x > 0.5 else 0 for x in LinearReg.predict(X_train)] )
y_pred_test_RL  = np.array([1 if x > 0.5 else 0 for x in LinearReg.predict(X_test)])
# Regressão Logística
y_pred_train_RLog = LogisticReg.predict(X_train)
y_pred_test_RLog  = LogisticReg.predict(X_test)
# Redes Neurais
y_pred_train_RNA = RNA.predict(X_train)
y_pred_test_RNA = RNA.predict(X_test)


#---------------------------------------------------------------------------
## Cálcula e mostra a Acurácia dos modelos
#---------------------------------------------------------------------------
from sklearn import metrics
print()
print('----------------------------------------------------------------------------')
print('----------     ACURÁCIA     ------------------------------------------------')
print('----------------------------------------------------------------------------')
print('Acurácia Árvore de Decisão:   ',metrics.accuracy_score(y_test, y_pred_test_DT))
print('Acurácia Regressão Linear:    ',metrics.accuracy_score(y_test, y_pred_test_RL))
print('Acurácia Regressão Logística: ',metrics.accuracy_score(y_test, y_pred_test_RLog))
print('Acurácia Redes Neurais:       ',metrics.accuracy_score(y_test, y_pred_test_RNA))
print('----------------------------------------------------------------------------')
print()
#---------------------------------------------------------------------------
## Mostra a Acurácia dos modelos
#---------------------------------------------------------------------------
print()
print('----------------------------------------------------------------------------')
print('----------     MATRIZ DE CONFUSÃO    ---------------------------------------')
print('----------------------------------------------------------------------------')
print('--  Árvore de Decisão  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_DT, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Regressão Linear  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RL, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Regressão Logística  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RLog, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Redes Neurais  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RNA, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print()

# relatório do modelo
'''Além de mostrar as métricas para a categoria alvo, 
o relatório mostra os valores para a categoria 0 também. 
Além disso, temos a métrica do f1-score, que é uma média harmônica entre a precisão e o recall.
O f1-score será alto quando ambas as métricas forem altas e similares, 
ou seja, o f1-score é maior quando há um “meio termo” entre precisão e recall.
'''
from sklearn.metrics import classification_report
print('----------------------------------------------------------------------------')
print('Relatório de classificação:\n', classification_report(y_test, y_pred_test_RLog, digits=4))
print('----------------------------------------------------------------------------')



#---------------------------------------------------------------------------
## Previsão treinamento e teste - REGRESSÂO
#---------------------------------------------------------------------------
# Árvore de Decisão
y_pred_train_DT_R  = dtree.predict_proba(X_train)[:,1]
y_pred_test_DT_R  = dtree.predict_proba(X_test)[:,1]

# Regressão Linear
y_pred_train_RL_R = LinearReg.predict(X_train)
y_pred_test_RL_R  = LinearReg.predict(X_test)
# Regressão Logística
y_pred_train_RLog_R = LogisticReg.predict_proba(X_train)[:,1]
y_pred_test_RLog_R  = LogisticReg.predict_proba(X_test)[:,1]
# Redes Neurais
y_pred_train_RNA_R = RNA.predict_proba(X_train)[:,1]
y_pred_test_RNA_R  = RNA.predict_proba(X_test)[:,1]

#---------------------------------------------------------------------------
## Cálcula e mostra RMSE dos modelos
#---------------------------------------------------------------------------

from math import sqrt
print()
print('----------------------------------------------------------------------------')
print('----------     RMSE ERROR    -----------------------------------------------')
print('----------------------------------------------------------------------------')
print('Árvore de Decisão:  ',  sqrt(np.mean((y_test - y_pred_test_DT_R) **2) ))
print('Regressão Linear:   ',  sqrt(np.mean((y_pred_test_RL_R -  y_test) ** 2) ))
print('Regressão Logística:',  np.mean((y_pred_test_RLog_R - y_test) ** 2) ** 0.5)
print('Redes Neurais:      ',  np.mean((y_pred_test_RNA_R - y_test) ** 2) ** 0.5)
print('----------------------------------------------------------------------------')
print()

#---------------------------------------------------------------------------
## Cálcula o KS2
#---------------------------------------------------------------------------
print()
print('----------------------------------------------------------------------------')
print('----------------     KS2    ------------------------------------------------')
print('----------------------------------------------------------------------------')
print('Árvore de Decisão:   ',du.KS2(y_test,y_pred_test_DT_R))
print('Regressão Linear:    ',du.KS2(y_test,y_pred_test_RL_R))
print('Regressão Logística: ',du.KS2(y_test,y_pred_test_RLog_R))
print('Redes Neurais:       ',du.KS2(y_test,y_pred_test_RNA_R))
print('----------------------------------------------------------------------------')
print()

#----------------------------------------------------------------------
## Montando um Data Frame (Matriz) com os resultados
#----------------------------------------------------------------------
# Conjunto de treinamento
df_train = pd.DataFrame(y_pred_train_DT_R, columns=['REGRESSION_DT'])
df_train['CLASSIF_DT'] = y_pred_train_DT
df_train['REGRESSION_RL'] = y_pred_train_RL_R
df_train['CLASSIF_RL'] =  [1 if x > 0.5 else 0 for x in y_pred_train_RL]
df_train['REGRESSION_RLog'] = y_pred_train_RLog_R
df_train['CLASSIF_RLog'] = y_pred_train_RLog
df_train['REGRESSION_RNA'] = y_pred_train_RNA_R
df_train['CLASSIF_RNA'] = y_pred_train_RNA
df_train['ALVO'] = [x for x in y_train]
df_train['TRN_TST'] = 'TRAIN'

# Conjunto de test
df_test = pd.DataFrame(y_pred_test_DT_R, columns=['REGRESSION_DT'])
df_test['CLASSIF_DT'] = y_pred_test_DT
df_test['REGRESSION_RL'] = y_pred_test_RL_R
df_test['CLASSIF_RL'] =  [1 if x > 0.5 else 0 for x in y_pred_test_RL]
df_test['REGRESSION_RLog'] = y_pred_test_RLog_R
df_test['CLASSIF_RLog'] = y_pred_test_RLog
df_test['REGRESSION_RNA'] = y_pred_test_RNA_R
df_test['CLASSIF_RNA'] = y_pred_test_RNA
df_test['ALVO'] = [x for x in y_test]
df_test['TRN_TST'] = 'TEST' 

print()
print('----------------    INÍCIO DA EXPORTAÇÃO RESULTADOS   ----------------------------------')
# Juntando Conjunto de Teste e Treinamento
df_total = pd.concat([df_test, df_train], sort = False)

## Exportando os dados para avaliação dos resultados em outra ferramenta
df_total.to_csv('resultado_comparacao2.csv')
print('----------------     FIM DA EXPORTAÇÃO RESULTADOS   ------------------------------------')





'''
#---------------------------------------------------------------------------
## Mostra as fórmulas das regressões para implantação em SQL Server, se precisar
#---------------------------------------------------------------------------

print()
print('----------------   REPRESENTAÇÃO DOS MODELOS EM SQL SERVER  ----------------')
du.LogisticFormulaSQL(LogisticReg, X_train)

print()
du.LinearFormulaSQL(LinearReg, X_train)
print('----------------------------------------------------------------------------')
'''



----------------     INÍCIO DE CARGA DE DADOS   ------------------------------------
-----------------     FIM DA CARGA DE DADOS   --------------------------------------
-> DADOS PARA ANÁLISE EXPORTADOS

----------------     SELEÇÃO DE VARIÁVEIS--------------------------------------
Num Features: 16
             -> pre_restricao_interna              
             -> pre_alto                           
             -> pre_medio                          
             -> pre_baixo                          
             -> pre_viuvo                          
             -> pre_solteiro                       
             -> pre_divorciado_nao_informado       
             -> pre_tipo_cliente_vip               
             -> pre_tipo_cliente_classico          
             -> pre_idade                          
             -> pre_qtde_produtos_pf_12m           
             -> pre_vl_totl_rend                   
             -> pre_uf_rs                          
             -> pre_in

"\n#---------------------------------------------------------------------------\n## Mostra as fórmulas das regressões para implantação em SQL Server, se precisar\n#---------------------------------------------------------------------------\n\nprint()\nprint('----------------   REPRESENTAÇÃO DOS MODELOS EM SQL SERVER  ----------------')\ndu.LogisticFormulaSQL(LogisticReg, X_train)\n\nprint()\ndu.LinearFormulaSQL(LinearReg, X_train)\nprint('----------------------------------------------------------------------------')\n"

In [17]:
#listando as variáveis dummys
X_test

,pre_restricao_interna,pre_alto,pre_medio,pre_baixo,pre_viuvo,pre_solteiro,pre_divorciado_nao_informado,pre_tipo_cliente_vip,pre_tipo_cliente_classico,pre_idade,pre_qtde_produtos_pf_12m,pre_vl_totl_rend,pre_uf_rs,pre_investidor,pre_neutro,pre_tomador
3282,0,0,0,1,0,1,0,0,0,0.967742,0.4,0.189683,1,1,0,0
6640,0,0,1,0,0,1,0,1,0,0.306452,0.6,0.345467,0,0,0,1
3567,0,0,0,1,0,1,0,0,1,0.064516,0.5,0.050000,0,1,0,0
3104,0,0,0,1,0,0,1,0,0,0.645161,0.3,0.166866,1,1,0,0
2381,0,0,1,0,0,0,1,0,0,0.322581,0.3,0.219935,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,1,0,1,0,0,1,0,1,0,0.241935,0.1,0.425000,0,0,0,1
5453,0,0,1,0,0,1,0,0,0,0.387097,0.4,0.000000,0,1,0,0
1734,0,0,0,1,0,1,0,0,1,0.177419,0.2,0.147137,0,1,0,0
6499,0,0,1,0,0,0,1,0,0,0.225806,0.4,0.232542,0,0,0,1


In [20]:
#---------------------------------------------------------------------------
## Cálcula e mostra a Acurácia dos modelos
#---------------------------------------------------------------------------
from sklearn import metrics
print()
print('----------------------------------------------------------------------------')
print('----------     ACURÁCIA     ------------------------------------------------')
print('----------------------------------------------------------------------------')
print('Acurácia Árvore de Decisão:   ',metrics.accuracy_score(y_test, y_pred_test_DT))
print('Acurácia Regressão Linear:    ',metrics.accuracy_score(y_test, y_pred_test_RL))
print('Acurácia Regressão Logística: ',metrics.accuracy_score(y_test, y_pred_test_RLog))
print('Acurácia Redes Neurais:       ',metrics.accuracy_score(y_test, y_pred_test_RNA))
print('----------------------------------------------------------------------------')
print()
#---------------------------------------------------------------------------
## Mostra a Acurácia dos modelos
#---------------------------------------------------------------------------
print()
print('----------------------------------------------------------------------------')
print('----------     MATRIZ DE CONFUSÃO    ---------------------------------------')
print('----------------------------------------------------------------------------')
print('--  Árvore de Decisão  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_DT, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Regressão Linear  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RL, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Regressão Logística  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RLog, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print('--  Redes Neurais  --')
print('----------------------------------------------------------------------------')
print(pd.crosstab(y_test, y_pred_test_RNA, rownames=['Real'], colnames=['Predito'], margins=True))
print('----------------------------------------------------------------------------')
print()

# relatório do modelo
'''Além de mostrar as métricas para a categoria alvo, 
o relatório mostra os valores para a categoria 0 também. 
Além disso, temos a métrica do f1-score, que é uma média harmônica entre a precisão e o recall.
O f1-score será alto quando ambas as métricas forem altas e similares, 
ou seja, o f1-score é maior quando há um “meio termo” entre precisão e recall.
'''
from sklearn.metrics import classification_report
print('----------------------------------------------------------------------------')
print('Relatório de classificação:\n', classification_report(y_test, y_pred_test_RLog, digits=4))
print('----------------------------------------------------------------------------')



----------------------------------------------------------------------------
----------     ACURÁCIA     ------------------------------------------------
----------------------------------------------------------------------------
Acurácia Árvore de Decisão:    0.8284
Acurácia Regressão Linear:     0.8504
Acurácia Regressão Logística:  0.8428
Acurácia Redes Neurais:        0.8388
----------------------------------------------------------------------------


----------------------------------------------------------------------------
----------     MATRIZ DE CONFUSÃO    ---------------------------------------
----------------------------------------------------------------------------
--  Árvore de Decisão  --
----------------------------------------------------------------------------
Predito     0    1   All
Real                    
0        1932  325  2257
1         104  139   243
All      2036  464  2500
----------------------------------------------------------------------------
-